# 04_gold_transform
Purpose: Build Gold business layer tables using cleaned Silver data.
- Daily revenue / order KPIs
- Product-level sales metrics
- Customer-level summaries
Author: Janak
Date: 2025-11-26

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from delta import *
import datetime

spark = SparkSession.builder.getOrCreate()

silver_base = "/tmp/delta/silver"
gold_base = "/tmp/delta/gold"

print("Silver base:", silver_base)
print("Gold base:", gold_base)


In [0]:
silver_tables = ["orders", "customers", "products", "payments", "inventory"]
silver_dfs = {}

for table in silver_tables:
    try:
        path = f"{silver_base}/{table}"
        df = spark.read.format("delta").load(path)
        silver_dfs[table] = df
        print(f"Loaded {table}, rows={df.count()}, cols={len(df.columns)}")
    except Exception as e:
        print(f"Skipping {table}: {e}")


In [0]:
for name, df in silver_dfs.items():
    df.createOrReplaceTempView(f"{name}_silver")
    print("Created view:", f"{name}_silver")



🥇 GOLD LAYER TABLES START HERE


GOLD Table 1: Daily Revenue (orders + payments)
Goal:

Show total revenue per day

Include # orders

Include # successful payments

In [0]:
daily_revenue = spark.sql("""
    SELECT
        to_date(o.order_date) AS order_date,
        COUNT(DISTINCT o.order_id) AS total_orders,
        SUM(o.amount) AS total_order_amount,
        SUM(CASE WHEN p.status = 'confirmed' THEN p.amount ELSE 0 END) AS total_confirmed_payments
    FROM orders_silver o
    LEFT JOIN payments_silver p
        ON o.order_id = p.order_id
    GROUP BY to_date(o.order_date)
    ORDER BY order_date
""")

display(daily_revenue)


In [0]:
gold_path = f"{gold_base}/daily_revenue"
daily_revenue.write.format("delta").mode("overwrite").partitionBy("order_date").save(gold_path)
print("✔ Gold table written:", gold_path)

daily_revenue.createOrReplaceTempView("daily_revenue_gold")


GOLD Table 2: Product Sales Summary
Goal:

Revenue per product

Quantity sold

Category performance

We compute quantity sold as:
If order exists → count 1 piece per order for simplicity
(You can replace later with real order_items table.)

In [0]:
product_sales = spark.sql("""
    SELECT
        p.product_id,
        p.product_name,
        p.category,
        COUNT(o.order_id) AS total_units_sold,
        SUM(o.amount) AS total_revenue
    FROM products_silver p
    LEFT JOIN orders_silver o
        ON p.product_id = substring(o.order_id, 2, 3) -- TEMP LOGIC for demo (update with real key later)
    GROUP BY p.product_id, p.product_name, p.category
    ORDER BY total_revenue DESC
""")

display(product_sales)


In [0]:
gold_path = f"{gold_base}/product_sales"
product_sales.write.format("delta").mode("overwrite").save(gold_path)
print("✔ Gold table written:", gold_path)

product_sales.createOrReplaceTempView("product_sales_gold")


GOLD Table 3: Customer Lifetime Value (CLV)
Goal:

Total spend per customer

Number of orders

Average order value

Loyalty tier

In [0]:
customer_ltv = spark.sql("""
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        c.loyalty_tier,
        COUNT(o.order_id) AS total_orders,
        SUM(o.amount) AS total_spent,
        AVG(o.amount) AS avg_order_value,
        MIN(o.order_date) AS first_order_date,
        MAX(o.order_date) AS last_order_date
    FROM customers_silver c
    LEFT JOIN orders_silver o
        ON c.customer_id = o.customer_id
    GROUP BY 
        c.customer_id, c.first_name, c.last_name, c.loyalty_tier
    ORDER BY total_spent DESC
""")

display(customer_ltv)


In [0]:
gold_path = f"{gold_base}/customer_ltv"
customer_ltv.write.format("delta").mode("overwrite").save(gold_path)
print("✔ Gold table written:", gold_path)

customer_ltv.createOrReplaceTempView("customer_ltv_gold")


GOLD Table 4: Payment Insights
Goal:

Payment success rate

Refund rate

Payment method usage

In [0]:
payment_summary = spark.sql("""
    SELECT
        payment_method,
        COUNT(payment_id) AS total_payments,
        SUM(CASE WHEN status='confirmed' THEN 1 ELSE 0 END) AS confirmed_count,
        SUM(CASE WHEN status='refunded' THEN 1 ELSE 0 END) AS refunded_count,
        SUM(CASE WHEN status='pending' THEN 1 ELSE 0 END) AS pending_count,
        SUM(amount) AS total_amount
    FROM payments_silver
    GROUP BY payment_method
    ORDER BY total_amount DESC
""")

display(payment_summary)


In [0]:
gold_path = f"{gold_base}/payment_summary"
payment_summary.write.format("delta").mode("overwrite").save(gold_path)
print("✔ Gold table written:", gold_path)

payment_summary.createOrReplaceTempView("payment_summary_gold")


In [0]:
kpi = spark.sql("""
SELECT
    (SELECT SUM(total_order_amount) FROM daily_revenue_gold) AS total_revenue,
    (SELECT SUM(total_orders) FROM daily_revenue_gold) AS total_orders,
    (SELECT COUNT(*) FROM customer_ltv_gold) AS total_customers,
    (SELECT COUNT(*) FROM product_sales_gold) AS total_products
""")

display(kpi)


In [0]:
for table in ["daily_revenue", "product_sales", "customer_ltv", "payment_summary"]:
    path = f"{gold_base}/{table}"
    try:
        df = spark.read.format("delta").load(path)
        print(f"✔ Gold table: {table}, Rows = {df.count()}")
    except Exception as e:
        print(f"❌ Missing Gold table: {table} — {e}")


In [0]:
manifest = {
    "run_time": datetime.datetime.now().isoformat(),
    "gold_tables": [
        {"name": "daily_revenue"},
        {"name": "product_sales"},
        {"name": "customer_ltv"},
        {"name": "payment_summary"},
    ]
}

manifest_path = "/dbfs/tmp/gold_manifest.json"
import json
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

print("Gold manifest saved to:", manifest_path)
